In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, mean_absolute_error, f1_score
import os
import json
import gc
from numba import jit
from tqdm import tqdm_notebook
from tqdm import tqdm
import catboost as cbt
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import KernelRidge
from scipy.signal import hilbert, hann, convolve
from scipy import stats
import scipy.spatial.distance as dist
from collections import Counter
from statistics import mode
import warnings
import math
from itertools import product
import ast
warnings.filterwarnings('ignore')

# Baseline1

In [2]:
path = r'./data/'
train = pd.read_csv(filepath_or_buffer=path + 'train_set.csv')
test = pd.read_csv(filepath_or_buffer=path + 'test_set.csv')
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25317 entries, 0 to 25316
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         25317 non-null  int64 
 1   age        25317 non-null  int64 
 2   job        25317 non-null  object
 3   marital    25317 non-null  object
 4   education  25317 non-null  object
 5   default    25317 non-null  object
 6   balance    25317 non-null  int64 
 7   housing    25317 non-null  object
 8   loan       25317 non-null  object
 9   contact    25317 non-null  object
 10  day        25317 non-null  int64 
 11  month      25317 non-null  object
 12  duration   25317 non-null  int64 
 13  campaign   25317 non-null  int64 
 14  pdays      25317 non-null  int64 
 15  previous   25317 non-null  int64 
 16  poutcome   25317 non-null  object
 17  y          25317 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 3.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10852 

In [14]:
# 数据描述（只针对数值型数据）

In [15]:
train.describe()

,ID,age,balance,day,duration,campaign,pdays,previous,y
count,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000
mean,12659.000000,40.935379,1357.555082,15.835289,257.732393,2.772050,40.248766,0.591737,0.116957
std,7308.532719,10.634289,2999.822811,8.319480,256.975151,3.136097,100.213541,2.568313,0.321375
min,1.000000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000
25%,6330.000000,33.000000,73.000000,8.000000,103.000000,1.000000,-1.000000,0.000000,0.000000
50%,12659.000000,39.000000,448.000000,16.000000,181.000000,2.000000,-1.000000,0.000000,0.000000
75%,18988.000000,48.000000,1435.000000,21.000000,317.000000,3.000000,-1.000000,0.000000,0.000000
max,25317.000000,95.000000,102127.000000,31.000000,3881.000000,55.000000,854.000000,275.000000,1.000000


In [16]:
test.describe()

,ID,age,balance,day,duration,campaign,pdays,previous
count,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000
mean,30743.500000,41.040638,1393.157298,15.680151,257.206137,2.770365,39.954755,0.546443
std,3132.846895,10.652369,3318.497101,8.302317,250.480906,3.063481,99.524056,1.805938
min,25318.000000,18.000000,-2604.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,28030.750000,33.000000,72.000000,8.000000,102.000000,1.000000,-1.000000,0.000000
50%,30743.500000,39.000000,450.000000,16.000000,181.000000,2.000000,-1.000000,0.000000
75%,33456.250000,49.000000,1440.250000,21.000000,322.000000,3.000000,-1.000000,0.000000
max,36169.000000,94.000000,81204.000000,31.000000,3102.000000,58.000000,871.000000,51.000000


In [17]:
# 训练集相关性检验，注意高比例的变量，要对这个特征进行深挖，做数值特征和类别特征的交互。

In [18]:
train.corr()

,ID,age,balance,day,duration,campaign,pdays,previous,y
ID,1.000000,0.008465,0.032719,-0.020171,0.229149,-0.038265,0.065807,0.047028,0.556627
age,0.008465,1.000000,0.093740,-0.016070,0.000416,0.006171,-0.026431,0.006575,0.029916
balance,0.032719,0.093740,1.000000,0.010245,0.026042,-0.010419,0.001032,0.015792,0.057564
day,-0.020171,-0.016070,0.010245,1.000000,-0.031946,0.168830,-0.092892,-0.050706,-0.031886
duration,0.229149,0.000416,0.026042,-0.031946,1.000000,-0.087780,0.000040,0.001315,0.394746
campaign,-0.038265,0.006171,-0.010419,0.168830,-0.087780,1.000000,-0.089224,-0.031667,-0.075173
pdays,0.065807,-0.026431,0.001032,-0.092892,0.000040,-0.089224,1.000000,0.411688,0.107565
previous,0.047028,0.006575,0.015792,-0.050706,0.001315,-0.031667,0.411688,1.000000,0.088337
y,0.556627,0.029916,0.057564,-0.031886,0.394746,-0.075173,0.107565,0.088337,1.000000


In [19]:
train.job.value_counts()  # 查看job的count数

blue-collar      5456
management       5296
technician       4241
admin.           2909
services         2342
retired          1273
self-employed     884
entrepreneur      856
unemployed        701
housemaid         663
student           533
unknown           163
Name: job, dtype: int64

In [20]:
# 一般需要将训练集和测试集合在一起

In [5]:
data = pd.concat([train, test])

In [6]:
# 把测试集的label用-1去标记，以便区别

In [7]:
test['y'] = -1
print(len(test.columns))

18


In [8]:
# 去除索引(不然在catboost时可能会报错)

In [9]:
data = train.append(test).reset_index(drop=True)
print(data.head())

ID  age         job   marital  education default  balance housing loan  \
0   1   43  management   married   tertiary      no      291     yes   no   
1   2   42  technician  divorced    primary      no     5076     yes   no   
2   3   47      admin.   married  secondary      no      104     yes  yes   
3   4   28  management    single  secondary      no     -994     yes  yes   
4   5   42  technician  divorced  secondary      no     2974     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome  y  
0   unknown    9   may       150         2     -1         0  unknown  0  
1  cellular    7   apr        99         1    251         2    other  0  
2  cellular   14   jul        77         2     -1         0  unknown  0  
3  cellular   18   jul       174         2     -1         0  unknown  0  
4   unknown   21   may       187         5     -1         0  unknown  0


In [26]:
# 筛选出类别特征做label encoder

In [10]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['ID', 'y']]
# 第一次运行
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
    data[i] = lbl.fit_transform(data[i].astype(str))  # 类别转化为数值

In [7]:
# 第二次指定类别时运行
for i in cat_col:
    data[i] = data[i].astype('category')

In [11]:
data.head()  # label encoder

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,4,1,2,0,291,1,0,2,9,8,150,2,-1,0,3,0
1,2,42,9,0,0,0,5076,1,0,0,7,0,99,1,251,2,1,0
2,3,47,0,1,1,0,104,1,1,0,14,5,77,2,-1,0,3,0
3,4,28,4,2,1,0,-994,1,1,0,18,5,174,2,-1,0,3,0
4,5,42,9,0,1,0,2974,1,0,2,21,8,187,5,-1,0,3,0


In [8]:
data.head()  # 指定类别特征后

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [9]:
data.info()  # category

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36169 entries, 0 to 36168
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   ID         36169 non-null  int64   
 1   age        36169 non-null  int64   
 2   job        36169 non-null  category
 3   marital    36169 non-null  category
 4   education  36169 non-null  category
 5   default    36169 non-null  category
 6   balance    36169 non-null  int64   
 7   housing    36169 non-null  category
 8   loan       36169 non-null  category
 9   contact    36169 non-null  category
 10  day        36169 non-null  int64   
 11  month      36169 non-null  category
 12  duration   36169 non-null  int64   
 13  campaign   36169 non-null  int64   
 14  pdays      36169 non-null  int64   
 15  previous   36169 non-null  int64   
 16  poutcome   36169 non-null  category
 17  y          36169 non-null  int64   
dtypes: category(9), int64(9)
memory usage: 2.8 MB


In [29]:
# 筛选出特征

In [10]:
feats = [i for i in data.columns if i not in ['ID', 'y']]
feats

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome']

In [31]:
len(feats)

16

In [32]:
# 预设model

In [11]:
model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=30, reg_alpha=0, reg_lambda=0.,
                        max_depth=-1, n_estimators=1500, objective='binary', metric='auc', subsample=0.95,
                        colsample_bytree=0.7, subsample_freq=1, learning_rate=0.02, random_state=2019)

In [12]:
data1 = data[data['y'] != -1][feats]
label1 = data[data['y'] != -1]['y']
testx = data[data['y'] == -1][feats]

In [13]:
train_x, vali_x, train_y, vali_y = train_test_split(data1, label1, test_size=0.3, random_state=42)

In [14]:
model.fit(train_x, train_y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
               importance_type='split', learning_rate=0.02, max_depth=-1,
               metric='auc', min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=1500, n_jobs=-1, num_leaves=30,
               objective='binary', random_state=2019, reg_alpha=0,
               reg_lambda=0.0, silent=True, subsample=0.95,
               subsample_for_bin=200000, subsample_freq=1)

In [49]:
model.predict_proba(testx)

array([[0.96819222, 0.03180778],
       [0.9971349 , 0.0028651 ],
       [0.99861994, 0.00138006],
       ...,
       [0.9987676 , 0.0012324 ],
       [0.99797424, 0.00202576],
       [0.98756099, 0.01243901]])

In [50]:
model.predict_proba(testx)[:, 1]

array([0.03180778, 0.0028651 , 0.00138006, ..., 0.0012324 , 0.00202576,
       0.01243901])

In [ ]:
# 预测出类别的概率

In [51]:
test_pre = model.predict_proba(testx)[:, 1]

In [ ]:
# 直接预测出类别（因为模型默人大于0.5为1，而有的时候并不一定是这个阈值，所以一般是输出概率值）

In [52]:
model.predict(testx)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [53]:
pre = data[data['y'] == -1][['ID']]
pre['pred'] = test_pre
len(pre)

10852

In [54]:
pre.head()

,ID,pred
25317,25318,0.031808
25318,25319,0.002865
25319,25320,0.001380
25320,25321,0.830813
25321,25322,0.026146


In [56]:
pre.to_csv(r'./result/baseline1.csv', index=False)  # 线上成绩0.9263011190601801

# Baseline2

In [57]:
# 5折交叉验证

In [15]:
n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

In [16]:
train_x = data[data['y'] != -1][feats]
train_y = data[data['y'] != -1]['y']

In [17]:
res = data[data['y'] == -1][['ID']]
res['pred'] = 0

In [18]:
res.head()

,ID,pred
25317,25318,0
25318,25319,0
25319,25320,0
25320,25321,0
25321,25322,0


In [19]:
test_x = data[data['y'] == -1][feats]
for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    model.fit(train_x1, train_y1, eval_set=[(train_x1, train_y1), (test_x1, test_y1)], 
        eval_metric='acc', early_stopping_rounds=100)    
    res['pred'] += model.predict_proba(test_x)[:, 1]

s auc: 0.952594	valid_1's auc: 0.934451
[162]	training's auc: 0.95267	valid_1's auc: 0.934483
[163]	training's auc: 0.952686	valid_1's auc: 0.934481
[164]	training's auc: 0.95277	valid_1's auc: 0.934471
[165]	training's auc: 0.952913	valid_1's auc: 0.934519
[166]	training's auc: 0.953063	valid_1's auc: 0.934572
[167]	training's auc: 0.95324	valid_1's auc: 0.934652
[168]	training's auc: 0.953366	valid_1's auc: 0.934717
[169]	training's auc: 0.953437	valid_1's auc: 0.934706
[170]	training's auc: 0.953478	valid_1's auc: 0.934704
[171]	training's auc: 0.953595	valid_1's auc: 0.934852
[172]	training's auc: 0.953628	valid_1's auc: 0.934802
[173]	training's auc: 0.953697	valid_1's auc: 0.934763
[174]	training's auc: 0.953769	valid_1's auc: 0.934759
[175]	training's auc: 0.953861	valid_1's auc: 0.934769
[176]	training's auc: 0.953926	valid_1's auc: 0.934764
[177]	training's auc: 0.953995	valid_1's auc: 0.934795
[178]	training's auc: 0.954088	valid_1's auc: 0.934855
[179]	training's auc: 0.9541

KeyboardInterrupt: 

In [47]:
res['pred'] = res['pred'] / n_splits
res.to_csv('./result/baseline2_kf5.csv', index=False)  # 线上成绩0.9328784942896541

# lgb指定类别(即不用label encoder，直接指定类别)

In [ ]:
# 起到了一定的类别特征

In [20]:
test_x = data[data['y'] == -1][feats]
for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    model.fit(train_x1, train_y1, eval_set=[(train_x1, train_y1), (test_x1, test_y1)], 
        categorical_feature=cat_col, eval_metric='acc', early_stopping_rounds=100)    
    res['pred'] += model.predict_proba(test_x)[:, 1]

 auc: 0.930569
[315]	training's auc: 0.965893	valid_1's auc: 0.930561
[316]	training's auc: 0.965958	valid_1's auc: 0.930567
[317]	training's auc: 0.966016	valid_1's auc: 0.930586
[318]	training's auc: 0.966075	valid_1's auc: 0.930555
[319]	training's auc: 0.966127	valid_1's auc: 0.93056
[320]	training's auc: 0.966201	valid_1's auc: 0.930552
[321]	training's auc: 0.966247	valid_1's auc: 0.930586
[322]	training's auc: 0.966311	valid_1's auc: 0.930665
[323]	training's auc: 0.966368	valid_1's auc: 0.930729
[324]	training's auc: 0.966457	valid_1's auc: 0.930741
[325]	training's auc: 0.966523	valid_1's auc: 0.930745
[326]	training's auc: 0.966595	valid_1's auc: 0.930727
[327]	training's auc: 0.966684	valid_1's auc: 0.930736
[328]	training's auc: 0.966752	valid_1's auc: 0.930752
[329]	training's auc: 0.966804	valid_1's auc: 0.930749
[330]	training's auc: 0.966844	valid_1's auc: 0.930761
[331]	training's auc: 0.9669	valid_1's auc: 0.930753
[332]	training's auc: 0.966982	valid_1's auc: 0.93074

In [21]:
res['pred'] = res['pred'] / n_splits
res.to_csv('./result/baseline2_kf5_cat_col.csv', index=False)  # 线上成绩0.93505728424523